In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
from math import *
import time

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
# defination of activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [5]:
# Build ResNet with three blocks
class net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [6]:
dimension = 8

In [7]:
# Defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
# Defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
# Generate points by random
data_size = 1000
def generate_data():
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x.cuda()

In [10]:
# Defination of loss function for DGM
def loss_function(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1])
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + u_hat**3 - f_temp)**2)/x.size()[0]
    
    Nb = 200
    y = (2*torch.rand(Nb, dimension)-1).cuda()
    y_norm = torch.norm(y, dim = 1).cuda()
    
    for i in range(dimension):
        y[:, i] = y[:, i] / y_norm
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 400.0
    return part_1 + lambda1 * part_2 

In [11]:
# Parameters initialization by normal distribution for neural networks
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = net(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

net(
  (layer_in): Linear(in_features=8, out_features=16, bias=True)
  (layer1): Linear(in_features=16, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=16, bias=True)
  (layer4): Linear(in_features=16, out_features=16, bias=True)
  (layer5): Linear(in_features=16, out_features=16, bias=True)
  (layer6): Linear(in_features=16, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
# Defination of relative error
def relative_error():
    x = generate_data()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [16]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = generate_data()
    x.requires_grad = True
    loss = loss_function(x)
    loss.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_8D_Dirichlet_nonlinear_swish.npy", error_save)
np.save("DGM_relative_error_8D_Dirichlet_nonlinear_swish.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(120396.5997, device='cuda:0')
current relative error is:  tensor(14.6861, device='cuda:0')
current epoch is:  50
current loss is:  tensor(6056.4452, device='cuda:0')
current relative error is:  tensor(6.3746, device='cuda:0')
current epoch is:  100
current loss is:  tensor(3471.7590, device='cuda:0')
current relative error is:  tensor(5.7906, device='cuda:0')
current epoch is:  150
current loss is:  tensor(1858.2963, device='cuda:0')
current relative error is:  tensor(4.9604, device='cuda:0')
current epoch is:  200
current loss is:  tensor(1813.8558, device='cuda:0')
current relative error is:  tensor(4.2410, device='cuda:0')
current epoch is:  250
current loss is:  tensor(1244.3667, device='cuda:0')
current relative error is:  tensor(3.8359, device='cuda:0')
current epoch is:  300
current loss is:  tensor(916.4842, device='cuda:0')
current relative error is:  tensor(3.7273, device='cuda:0')
current epoch is:  350
current loss is:  tensor(8

current epoch is:  3050
current loss is:  tensor(30.6404, device='cuda:0')
current relative error is:  tensor(0.8415, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(30.9111, device='cuda:0')
current relative error is:  tensor(0.8042, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(33.7335, device='cuda:0')
current relative error is:  tensor(0.8275, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(30.0692, device='cuda:0')
current relative error is:  tensor(0.8800, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(30.4014, device='cuda:0')
current relative error is:  tensor(0.8126, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(32.5183, device='cuda:0')
current relative error is:  tensor(0.7958, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(29.7897, device='cuda:0')
current relative error is:  tensor(0.8586, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(29.400

current epoch is:  6100
current loss is:  tensor(10.2168, device='cuda:0')
current relative error is:  tensor(0.4584, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(9.8107, device='cuda:0')
current relative error is:  tensor(0.5166, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(9.2650, device='cuda:0')
current relative error is:  tensor(0.4936, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(10.0105, device='cuda:0')
current relative error is:  tensor(0.5182, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(10.8032, device='cuda:0')
current relative error is:  tensor(0.4797, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(9.4088, device='cuda:0')
current relative error is:  tensor(0.4872, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(10.0730, device='cuda:0')
current relative error is:  tensor(0.4479, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(9.9088, d

current epoch is:  9200
current loss is:  tensor(2.7842, device='cuda:0')
current relative error is:  tensor(0.2285, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(2.5589, device='cuda:0')
current relative error is:  tensor(0.2376, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(2.7308, device='cuda:0')
current relative error is:  tensor(0.2361, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(2.8277, device='cuda:0')
current relative error is:  tensor(0.2163, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(2.4095, device='cuda:0')
current relative error is:  tensor(0.2072, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(2.3992, device='cuda:0')
current relative error is:  tensor(0.2170, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(2.3580, device='cuda:0')
current relative error is:  tensor(0.2106, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(2.4329, devic

current epoch is:  12250
current loss is:  tensor(0.5231, device='cuda:0')
current relative error is:  tensor(0.0984, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(0.5576, device='cuda:0')
current relative error is:  tensor(0.1004, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(0.5095, device='cuda:0')
current relative error is:  tensor(0.0888, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.4986, device='cuda:0')
current relative error is:  tensor(0.0892, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.4797, device='cuda:0')
current relative error is:  tensor(0.0921, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.4613, device='cuda:0')
current relative error is:  tensor(0.0875, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.4896, device='cuda:0')
current relative error is:  tensor(0.0841, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.459

current epoch is:  15300
current loss is:  tensor(0.1219, device='cuda:0')
current relative error is:  tensor(0.0431, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(0.1272, device='cuda:0')
current relative error is:  tensor(0.0644, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(0.1225, device='cuda:0')
current relative error is:  tensor(0.0458, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.1337, device='cuda:0')
current relative error is:  tensor(0.0456, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.1205, device='cuda:0')
current relative error is:  tensor(0.0469, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.1124, device='cuda:0')
current relative error is:  tensor(0.0456, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.1133, device='cuda:0')
current relative error is:  tensor(0.0416, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.103

current epoch is:  18350
current loss is:  tensor(0.0609, device='cuda:0')
current relative error is:  tensor(0.0368, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(0.0626, device='cuda:0')
current relative error is:  tensor(0.0323, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(0.0420, device='cuda:0')
current relative error is:  tensor(0.0279, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.0528, device='cuda:0')
current relative error is:  tensor(0.0345, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.0541, device='cuda:0')
current relative error is:  tensor(0.0290, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.0419, device='cuda:0')
current relative error is:  tensor(0.0364, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.0467, device='cuda:0')
current relative error is:  tensor(0.0291, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.038